In [ ]:
import zmq
import time

In [ ]:
class BoxPublisher:
    def __init__(self):
        context = zmq.Context()
        self.pub = context.socket(zmq.PUB)
        self.pub.bind("tcp://*:5555")
        self.rep = context.socket(zmq.REP)
        self.rep.bind("tcp://*:5556")
        self.active = True

    def close(self):
        self.rep.close()
        self.pub.close()
    
    def publish(self, messages, debug=False):
        if self.active:
            if debug:
                print("Sending messages " + str(messages))
            self.pub.send_multipart(messages)
            try:
                command = rep.recv(flags=zmq.NOBLOCK)
                if debug:
                    print("recvd TERMINATE command")
                self.rep.send(b"ACK")
                self.active = False
            except:
                pass
    

In [ ]:
if __name__ == "__main__":
    pub = BoxPublisher()
    while pub.active:
        time.sleep(1)
        messages = (b"Hello", b"World")
        pub.publish(messages, True)
    print("Shutting down")
    pub.close()